# COMP 47670 - A2 Task 1 

Data Collection

In [1]:
import json, requests, urllib
from pathlib import Path
import urllib.request
import bs4


Create directory for raw data storage, if it does not already exist:



In [2]:
dir_raw = Path("raw")
dir_raw.mkdir(parents=True, exist_ok=True)

Download homepage using BeautifulSoup

In [3]:
url_prefix = 'http://mlg.ucd.ie/modules/python/EVdata/'
response = urllib.request.urlopen(url_prefix)
page_soup = bs4.BeautifulSoup(response,"html.parser")
response.close()

Search all "a" tags in the body to compile list of routes and their corresponding html pages

In [4]:
atags = page_soup.body.ul.find_all("a")
route_pages = []
for a in atags:
    if a.has_attr('href'):
        route_pages.append(a['href'])
        

Download the gpx file from each html page into a dictionary

In [5]:
route_names = {}

for route in route_pages:
    #define endpoint for specific route
    endpoint = url_prefix+route
    #define key to be used in route_files
    key = route.split(".")[0]
    #open url for this route
    response = urllib.request.urlopen(endpoint)
    #beautify this page
    page_soup = bs4.BeautifulSoup(response,"html.parser")
    response.close()
    #save text name of route
    value = page_soup.h1.text
    #save to dictionary
    route_names[key] = value
    #get gpx file name
    gpx_file = page_soup.a['href']
    #download this file to raw directory
    out_path = dir_raw / (key+'.gpx')
    url = url_prefix+gpx_file
    urllib.request.urlretrieve(url, filename=out_path, reporthook=None, data=None)
    
    


Output route key value pairs to json as well

In [6]:
with open('route_names.json', 'w') as fp:
    json.dump(route_names, fp)